 <span>

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

</span>


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

- Interceptor: txt database
- we take till rank 200 and not first 200 rows
- find vs findall
- NYU issue
- Indian Institute of Science (IISc) Bangalore issue
- json['data'][0].keys()

In [3]:
def find_div_and_number(soup, class_name):
    
    div = soup.find('div', class_= class_name)
    if div is not None:
        return div.find('div', class_='number').text.replace('\n', '').replace(',', '')
    else: return np.NaN
        

def parse_uni_details(url):
    page_body = requests.get('https://topuniversities.com' + uni['url']).text

    soup = BeautifulSoup(page_body, "html.parser")
    
    faculty_total = find_div_and_number(soup, 'total faculty')
    faculty_intl  = find_div_and_number(soup, 'inter faculty')
    student_total  = find_div_and_number(soup, 'total student')
    student_intl  = find_div_and_number(soup, 'total inter')
    
    return faculty_total, faculty_intl, student_total, student_intl

In [4]:
#ex1.1 
r = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508153509501')
json = r.json()

topuniversities = pd.DataFrame()

for (idx, uni) in enumerate(json['data']):
    
    if int(uni['rank_display'].replace('=','')) > 200:
        break
        
    faculty_total, faculty_intl, student_total, student_intl = parse_uni_details(uni['url'])
        
    entry = pd.DataFrame({'name': uni['title'], 'rank': uni['rank_display'].replace('=', ''), 'country': uni['country'],
                          'region': uni['region'], 'faculty_intl': faculty_intl, 'faculty_total': faculty_total,
                          'student_intl': student_intl, 'student_total': student_total}, index=[idx]) 
    topuniversities = topuniversities.append(entry)

# Rearrange columns order
topuniversities = topuniversities[['name', 'rank', 'country', 'region', 'faculty_intl', 'faculty_total', 'student_intl', 'student_total']]

# Change rank, faculty and student cols to numeric value
topuniversities[['rank','faculty_intl', 'faculty_total', 'student_intl', 'student_total']]\
    = topuniversities[['rank','faculty_intl', 'faculty_total', 'student_intl', 'student_total']].apply(pd.to_numeric)
    
# Serialization as a pickle file
topuniversities.to_pickle('pickled/topuniversities.pkl')
    
topuniversities.head()

,name,rank,country,region,faculty_intl,faculty_total,student_intl,student_total
0,Massachusetts Institute of Technology (MIT),1,United States,North America,1679.0,2982.0,3717.0,11067.0
1,Stanford University,2,United States,North America,2042.0,4285.0,3611.0,15878.0
2,Harvard University,3,United States,North America,1311.0,4350.0,5266.0,22429.0
3,California Institute of Technology (Caltech),4,United States,North America,350.0,953.0,647.0,2255.0
4,University of Cambridge,5,United Kingdom,Europe,2278.0,5490.0,6699.0,18770.0


In [5]:
topuniversities = pd.read_pickle('pickled/topuniversities.pkl')
topuniversities.head()

,name,rank,country,region,faculty_intl,faculty_total,student_intl,student_total
0,Massachusetts Institute of Technology (MIT),1,United States,North America,1679.0,2982.0,3717.0,11067.0
1,Stanford University,2,United States,North America,2042.0,4285.0,3611.0,15878.0
2,Harvard University,3,United States,North America,1311.0,4350.0,5266.0,22429.0
3,California Institute of Technology (Caltech),4,United States,North America,350.0,953.0,647.0,2255.0
4,University of Cambridge,5,United Kingdom,Europe,2278.0,5490.0,6699.0,18770.0


Only NYU and IISc Bangalore have NaN values:

In [6]:
topuniversities[topuniversities.isnull().any(1)]

,name,rank,country,region,faculty_intl,faculty_total,student_intl,student_total
51,New York University (NYU),52,United States,North America,NaN,NaN,NaN,NaN
189,Indian Institute of Science (IISc) Bangalore,190,India,Asia,NaN,423.0,47.0,4071.0


- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?

In [7]:
#1.2 a
df_1a = topuniversities.loc[:,('name', 'rank', 'country', 'region', 'faculty_total', 'student_total')]
df_1a['ratio faculty/students'] = topuniversities.faculty_total / topuniversities.student_total
df_1a = df_1a.sort_values(['ratio faculty/students'], ascending= False)
df_1a[0:10]

,name,rank,country,region,faculty_total,student_total,ratio faculty/students
3,California Institute of Technology (Caltech),4,United States,North America,953.0,2255.0,0.422616
15,Yale University,16,United States,North America,4940.0,12402.0,0.398323
5,University of Oxford,6,United Kingdom,Europe,6750.0,19720.0,0.342292
4,University of Cambridge,5,United Kingdom,Europe,5490.0,18770.0,0.292488
16,Johns Hopkins University,17,United States,North America,4462.0,16146.0,0.276353
1,Stanford University,2,United States,North America,4285.0,15878.0,0.269870
0,Massachusetts Institute of Technology (MIT),1,United States,North America,2982.0,11067.0,0.269450
185,University of Rochester,186,United States,North America,2569.0,9636.0,0.266604
18,University of Pennsylvania,19,United States,North America,5499.0,20639.0,0.266437
17,Columbia University,18,United States,North America,6189.0,25045.0,0.247115


In [8]:
#1.2 b
df_1b = topuniversities.loc[:,('name', 'rank', 'country', 'region', 'student_intl', 'student_total')]
df_1b['ratio students_intl/students_total'] = topuniversities.student_intl / topuniversities.student_total
df_1b = df_1b.sort_values(['ratio students_intl/students_total'], ascending= False)
df_1b[0:10]


,name,rank,country,region,student_intl,student_total,ratio students_intl/students_total
34,London School of Economics and Political Scien...,35,United Kingdom,Europe,6748.0,9760.0,0.691393
11,Ecole Polytechnique Fédérale de Lausanne (EPFL),12,Switzerland,Europe,5896.0,10343.0,0.570047
7,Imperial College London,8,United Kingdom,Europe,8746.0,16090.0,0.543567
198,Maastricht University,200,Netherlands,Europe,8234.0,16385.0,0.502533
46,Carnegie Mellon University,47,United States,North America,6385.0,13356.0,0.478062
6,UCL (University College London),7,United Kingdom,Europe,14854.0,31080.0,0.477928
91,University of St Andrews,92,United Kingdom,Europe,4030.0,8800.0,0.457955
41,The University of Melbourne,41,Australia,Oceania,18030.0,42182.0,0.427434
126,Queen Mary University of London,127,United Kingdom,Europe,6806.0,16135.0,0.421816
25,The University of Hong Kong,26,Hong Kong,Asia,8230.0,20214.0,0.407144


- Answer the previous question aggregating the data by (c) country and (d) region.

In [9]:
def getRatioSFBy (source, by ) :
    res = topuniversities.groupby(by,as_index=False).sum()
    res['ratio faculty/student'] = res['faculty_total']/res['student_total']
    return res[[by,'ratio faculty/student']].sort_values('ratio faculty/student',ascending=False).dropna()

In [10]:
def getRatioInterBy (source, by ) :
    res = topuniversities.groupby(by,as_index=False).sum()
    res['ratio international student'] = res['student_intl']/res['student_total']
    return res[[by,'ratio international student']].sort_values('ratio international student',ascending=False).dropna()

In [11]:
#1.3 c ratio between faculty members and students
dataex2cSF = getRatioSFBy(topuniversities,'country')
dataex2cSF

,country,ratio faculty/student
23,Russia,0.221910
8,Denmark,0.177261
24,Saudi Arabia,0.175828
25,Singapore,0.161530
18,Malaysia,0.153893
17,Japan,0.152479
27,South Korea,0.141721
30,Switzerland,0.140434
32,United Kingdom,0.136962
15,Israel,0.136047


In [12]:
#1.3 c ratio of international students
dataex2cInter = getRatioInterBy(topuniversities,'country')
dataex2cInter

,country,ratio international student
1,Australia,0.352189
32,United Kingdom,0.341705
12,Hong Kong,0.310751
2,Austria,0.309980
30,Switzerland,0.302396
25,Singapore,0.276537
5,Canada,0.260161
21,New Zealand,0.258215
14,Ireland,0.235299
20,Netherlands,0.232980


In [13]:
#1.3 c ratio between faculty members and students
dataex2dSF = getRatioSFBy(topuniversities,'region')
dataex2dSF

,region,ratio faculty/student
1,Asia,0.132260
4,North America,0.117776
2,Europe,0.111564
3,Latin America,0.108657
0,Africa,0.088450
5,Oceania,0.072385


In [14]:
#1.3 d ratio of international students
dataex2dInter = getRatioInterBy(topuniversities,'region')
dataex2dInter

,region,ratio international student
5,Oceania,0.339261
2,Europe,0.229589
4,North America,0.188906
0,Africa,0.169703
1,Asia,0.136431
3,Latin America,0.087520


In [15]:
#ex2.1
r = requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
json = r.json()
json['data'][0]

{'aliases': 'University of Oxford',
 'location': 'United Kingdom',
 'member_level': '0',
 'name': 'University of Oxford',
 'nid': 468,
 'rank': '1',
 'rank_order': '10',
 'record_type': 'master_account',
 'scores_citations': '99.1',
 'scores_citations_rank': '15',
 'scores_industry_income': '63.7',
 'scores_industry_income_rank': '169',
 'scores_international_outlook': '95.0',
 'scores_international_outlook_rank': '24',
 'scores_overall': '94.3',
 'scores_overall_rank': '10',
 'scores_research': '99.5',
 'scores_research_rank': '1',
 'scores_teaching': '86.7',
 'scores_teaching_rank': '5',
 'stats_female_male_ratio': '46 : 54',
 'stats_number_students': '20,409',
 'stats_pc_intl_students': '38%',
 'stats_student_staff_ratio': '11.2',
 'subjects_offered': 'Archaeology,Art, Performing Arts & Design,Biological Sciences,Business & Management,Chemical Engineering,Chemistry,Civil Engineering,Computer Science,Economics & Econometrics,Electrical & Electronic Engineering,General Engineering,Geo

In [16]:
the = pd.DataFrame()

for (idx, uni) in enumerate(json['data']):
    
    if '201' in uni['rank']:
        break
                
    entry = pd.DataFrame({'name': uni['name'], 'rank': uni['rank'].replace('=', ''), 'country': uni['location'],\
                          'stats_student_staff_ratio': uni['stats_student_staff_ratio'],\
                          'stats_pc_intl_student': uni['stats_pc_intl_students'].replace('%', ''),\
                          'student_total': uni['stats_number_students'].replace(',', ''),\
                          'scores_international_outlook': uni['scores_international_outlook']}, index=[idx]) 
    the = the.append(entry)

# Rearrange columns order
the = the[['name', 'rank', 'country', 'stats_student_staff_ratio', 'stats_pc_intl_student', 'student_total', 'scores_international_outlook']]

# Change rank, faculty and student cols to numeric value
the[['rank','stats_student_staff_ratio', 'stats_pc_intl_student', 'student_total', 'scores_international_outlook']]\
    = the[['rank','stats_student_staff_ratio', 'stats_pc_intl_student', 'student_total', 'scores_international_outlook']].apply(pd.to_numeric)
    
# Serialization as a pickle file
the.to_pickle('pickled/the.pkl')
    
the.head()

,name,rank,country,stats_student_staff_ratio,stats_pc_intl_student,student_total,scores_international_outlook
0,University of Oxford,1,United Kingdom,11.2,38,20409,95.0
1,University of Cambridge,2,United Kingdom,10.9,35,18389,93.0
2,California Institute of Technology,3,United States,6.5,27,2209,59.7
3,Stanford University,3,United States,7.5,22,15845,77.6
4,Massachusetts Institute of Technology,5,United States,8.7,34,11177,87.6


In [17]:
the = pd.read_pickle('pickled/the.pkl')
the.head()

,name,rank,country,stats_student_staff_ratio,stats_pc_intl_student,student_total,scores_international_outlook
0,University of Oxford,1,United Kingdom,11.2,38,20409,95.0
1,University of Cambridge,2,United Kingdom,10.9,35,18389,93.0
2,California Institute of Technology,3,United States,6.5,27,2209,59.7
3,Stanford University,3,United States,7.5,22,15845,77.6
4,Massachusetts Institute of Technology,5,United States,8.7,34,11177,87.6


No entries has NaN values:

In [18]:
the.isnull().any(1).sum()

0

In [19]:
#2.1 (a)  ratio between faculty members and students THE
the[['name','rank','stats_student_staff_ratio']].sort_values(['stats_student_staff_ratio'], ascending= False)

,name,rank,stats_student_staff_ratio
99,University of Bonn,100,77.9
124,University of Mannheim,125,74.5
91,Technical University of Berlin,92,64.3
163,University of Erlangen-Nuremberg,162,58.5
78,RWTH Aachen University,79,57.7
87,Free University of Berlin,88,57.0
61,Humboldt University of Berlin,62,56.5
40,Technical University of Munich,41,54.4
144,University of Cologne,145,50.9
174,Université Libre de Bruxelles,175,43.4


In [20]:
#2.1 (b) ratio of international students
the[['name','rank','stats_pc_intl_student']].sort_values(['stats_pc_intl_student'], ascending= False)


,name,rank,stats_pc_intl_student
24,London School of Economics and Political Science,25,71
178,University of Luxembourg,179,57
37,École Polytechnique Fédérale de Lausanne,38,55
7,Imperial College London,8,55
102,Maastricht University,103,50
15,University College London,16,49
143,University of St Andrews,143,48
23,Carnegie Mellon University,24,45
120,Queen Mary University of London,121,45
39,University of Hong Kong,40,42


In [21]:
# 2.1  (c)  ratio between faculty members and students by country THE 
theRatioFSCount = the[['name','country','stats_student_staff_ratio']].groupby('country',as_index=False).mean().sort_values(['stats_student_staff_ratio'], ascending= False)
theRatioFSCount

,country,stats_student_staff_ratio
8,Germany,44.025000
2,Belgium,38.550000
0,Australia,27.462500
10,Ireland,22.800000
1,Austria,20.800000
3,Canada,19.716667
9,Hong Kong,19.300000
15,New Zealand,18.800000
16,Norway,18.600000
14,Netherlands,18.330769


In [22]:
# 2.1  (c) ratio of international students by country   THE
theRatioInterCount = the[['name','country','stats_pc_intl_student']].groupby('country',as_index=False).mean().sort_values(['stats_pc_intl_student'], ascending= False)
theRatioInterCount

,country,stats_pc_intl_student
13,Luxembourg,57.000000
25,United Kingdom,36.548387
9,Hong Kong,32.800000
23,Switzerland,31.428571
0,Australia,30.750000
18,Singapore,30.500000
15,New Zealand,29.000000
10,Ireland,27.000000
1,Austria,26.000000
3,Canada,22.500000


In [23]:
#create a dictionary country -> region from the topuniversities dataFrame because there is no region column for the THE
mapCountryToRegion = topuniversities[['country','region']].set_index('country').to_dict(orient='dict')['region']
#there is missing entry so we fill them
mapCountryToRegion['Russian Federation']='Europe'
mapCountryToRegion['Luxembourg']='Europe'

In [24]:
# 2.1  (d)  ratio of international students by country    THE 
theRatioFSRegion = theRatioFSCount.copy( )
theRatioFSRegion['region'] = theRatioFSRegion['country'].map(mapCountryToRegion)
theRatioFSRegion = theRatioFSRegion.groupby('region').mean().sort_values(['stats_student_staff_ratio'], ascending= False)
theRatioFSRegion

,stats_student_staff_ratio
region,
Oceania,23.131250
Europe,18.279042
North America,15.810753
Asia,13.398810
Africa,11.700000


In [25]:
# 2.1  (d) country ratio between faculty members and students by region    THE 
theRatioInterRegion = theRatioInterCount.copy( )
theRatioInterRegion['region'] = theRatioInterRegion['country'].map(mapCountryToRegion)
theRatioInterRegion = theRatioInterRegion.groupby('region').mean().sort_values(['stats_pc_intl_student'], ascending= False)
theRatioFSRegion

,stats_student_staff_ratio
region,
Oceania,23.131250
Europe,18.279042
North America,15.810753
Asia,13.398810
Africa,11.700000


In [26]:
#2.2 merge the dataFrames topuniversities and THE

cleanThe = the.copy()
#cleanThe['country'] = cleanThe['country'].map(lambda x : x.split('(')[0])

cleanThe ['name'] = cleanThe ['name'].map(lambda x : x.replace("’","'").replace('É' , 'E'))

cleanThe.shape

(200, 7)

In [27]:
cleanTopuniversities = topuniversities.copy()
#delete 'The ' at the begginning so that there is more match
cleanTopuniversities['name'] = cleanTopuniversities['name'].map(lambda x : x[4::] if (x[:4] == 'The ') else x.replace('É' , 'E') )
#delete the ' (###)' at the end of the name
cleanTopuniversities['name'] = cleanTopuniversities['name'].map(lambda x : x.split('(')[0][:-1] if (x[-1] == ')') else x )

#'UCL' : 'University College London'
cleanTopuniversities.loc[6,'name'] = 'University College London'
#ETH Zurich - Swiss Federal Institute of Technology -> ETH Zurich – Swiss Federal Institute of Technology Zurich 
cleanTopuniversities.loc[9,'name'] = 'ETH Zurich – Swiss Federal Institute of Technology Zurich'
#KAIST - Korea Advanced Institute of Science & Technology -> Korea Advanced Institute of Science and Technology (KAIST)
cleanTopuniversities.loc[40,'name'] = 'Korea Advanced Institute of Science and Technology (KAIST)'
#Ludwig-Maximilians-Universität München -> LMU Munich 
cleanTopuniversities.loc[65,'name'] = 'LMU Munich'
#Ruprecht-Karls-Universität Heidelberg -> Heidelberg University
cleanTopuniversities.loc[67,'name'] = 'Heidelberg University'
#Pohang University of Science And Technology -> Pohang University of Science and Technology
cleanTopuniversities.loc[71,'name'] = 'Pohang University of Science and Technology'
#University of North Carolina, Chapel Hill -> University of North Carolina at Chapel Hill 
cleanTopuniversities.loc[79,'name'] = 'University of North Carolina at Chapel Hill'
#Trinity College Dublin, The University of Dublin -> Trinity College Dublin
cleanTopuniversities.loc[87,'name'] = 'Trinity College Dublin'
#Washington University in St. Louis -> Washington University in St Louis
cleanTopuniversities.loc[99,'name'] = 'Washington University in St Louis'
#KIT, Karlsruhe Institute of Technology -> Karlsruhe Institute of Technology
cleanTopuniversities.loc[106,'name'] = 'Karlsruhe Institute of Technology'
#Sungkyunkwan University -> Sungkyunkwan University (SKKU)
cleanTopuniversities.loc[107,'name'] ='Sungkyunkwan University (SKKU)'
#Humboldt-Universität zu Berlin -> Humboldt University of Berlin
cleanTopuniversities.loc[119,'name'] ='Humboldt University of Berlin'
#Wageningen University -> Wageningen University & Research
cleanTopuniversities.loc[123,'name'] = 'Wageningen University & Research'
#Freie Universitaet Berlin -> Free University of Berlin
cleanTopuniversities.loc[124,'name'] = 'Free University of Berlin'
#Université de Montréal -> University of Montreal
cleanTopuniversities.loc[124,'name'] = 'University of Montreal'
#Université Pierre et Marie Curie -> Pierre and Marie Curie University
cleanTopuniversities.loc[130,'name'] = 'Pierre and Marie Curie University'
#Technische Universität Berlin -> Technical University of Berlin
cleanTopuniversities.loc[143,'name'] = 'Technical University of Berlin'
#Université catholique de Louvain -> Université Catholique de Louvain
cleanTopuniversities.loc[152,'name'] = 'Université Catholique de Louvain'
#Universitat de Barcelona -> Autonomous University of Barcelona
cleanTopuniversities.loc[155,'name'] = 'Autonomous University of Barcelona'
#Eberhard Karls Universität Tübingen -> University of Tübingen
cleanTopuniversities.loc[163,'name'] = 'University of Tübingen'
#Albert-Ludwigs-Universitaet Freiburg -> University of Freiburg
cleanTopuniversities.loc[170,'name'] = 'University of Freiburg'
#Vrije Universiteit Brussel -> Université Libre de Bruxelles
cleanTopuniversities.loc[181,'name'] = 'Université Libre de Bruxelles'
#Scuola Superiore Sant'Anna Pisa di Studi Universitari e di Perfezionamento -> Scuola Superiore Sant'Anna
cleanTopuniversities.loc[181,'name'] = "Scuola Superiore Sant'Anna"

cleanTopuniversities.shape

(199, 8)

In [28]:
mergeUni = cleanTopuniversities.set_index('name').join(other = cleanThe.set_index('name') , how = 'outer',lsuffix='_top uni', rsuffix='_THE')
mergeUni

,rank_top uni,country_top uni,region,faculty_intl,faculty_total,student_intl,student_total_top uni,rank_THE,country_THE,stats_student_staff_ratio,stats_pc_intl_student,student_total_THE,scores_international_outlook
name,,,,,,,,,,,,,
Aalto University,137.0,Finland,Europe,370.0,1257.0,1831.0,12147.0,190.0,Finland,20.2,20.0,12744.0,72.5
Aarhus University,119.0,Denmark,Europe,602.0,2316.0,3762.0,26226.0,109.0,Denmark,14.3,12.0,25167.0,75.3
Alma Mater Studiorum - University of Bologna,188.0,Italy,Europe,153.0,2990.0,4195.0,63399.0,NaN,NaN,NaN,NaN,NaN,NaN
Arizona State University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.0,United States,20.1,19.0,44475.0,55.2
Australian National University,20.0,Australia,Oceania,927.0,1600.0,5551.0,14442.0,48.0,Australia,19.3,35.0,15986.0,94.3
Autonomous University of Barcelona,156.0,Spain,Europe,108.0,3863.0,6134.0,46002.0,147.0,Spain,12.9,16.0,32309.0,60.1
Boston University,81.0,United States,North America,379.0,3157.0,7041.0,25662.0,70.0,United States,8.6,25.0,24833.0,60.0
Brown University,53.0,United States,North America,379.0,1303.0,1825.0,9251.0,50.0,United States,10.7,20.0,8898.0,59.0
California Institute of Technology,4.0,United States,North America,350.0,953.0,647.0,2255.0,3.0,United States,6.5,27.0,2209.0,59.7


In [29]:
listOkay = mergeUni.index.values.tolist()


iterTopuniversities = cleanTopuniversities.iterrows()


for itTop in iterTopuniversities :
    iterTheClassment = cleanThe.iterrows()
    topname = itTop[1]['name']
    splittopname = topname.split(' ')
    if ( not topname in listOkay) :
        for iterThe in iterTheClassment :
            thename = iterThe[1]['name']
            splitthename = thename.split(' ')
            if ((not thename in listOkay )and iterThe[1]['country']== itTop[1]['country'] ) :
                print(itTop[0],topname,"->",thename,'|')

           
                                     
        